# Predict Protein-Ligand 3D Interactions Using DiffDock NIM

This example notebook shows how to connect NVIDIA DiffDock NIM to perform protein-ligand docking. NVIDIA BioNeMo NIMS can be integrated into existing virtual screening workflows to leverage cutting edge Gen-AI capabilities for Drug discovery, from ligand generation to protein folding to docking. For more details, please visit the NVIDIA BioNeMo NIMS homepage at https://build.nvidia.com/explore/healthcare 

## 1. Get the Diffdock NIM up and running

Please visit [NVIDIA DiffDock NIM](https://build.nvidia.com/explore/healthcare?snippet_tab=Docker&self_hosted_api=true#diffdock) and [DiffDock NIMS documentation page](https://docs.nvidia.com/nim/bionemo/diffdock/latest/overview.html) for more details and examples.

### 1.1 Perform the following steps on the node intended for hosting the DiffDock NIM


1. Open a Terminal.
2. Run `docker login --username '$oauthtoken' --password $NGC_API_KEY nvcr.io`


### 1.2 Start here
3. Run the following command to download and start the DiffDock server. It will pull the docker container and the required model weight from NGC. 

```bash
    export NGC_API_KEY=<PASTE_API_KEY_HERE>

    docker run -it --rm \
        --runtime=nvidia -e NVIDIA_VISIBLE_DEVICES=0 \
        --shm-size=2G \
        --ulimit memlock=-1 \
        --ulimit stack=67108864 \
        -e NGC_API_KEY=$NGC_API_KEY \
        -p 8000:8000 \
        nvcr.io/nim/mit/diffdock:1.2.0
 ```

4. You may return to this notebook and start the following cells once you see a response like this on your terminal:

 ```
    I0421 21:04:17.461556 81 grpc_server.cc:2519] Started GRPCInferenceService at 0.0.0.0:8001
    I0421 21:04:17.461909 81 http_server.cc:4637] Started HTTPService at 0.0.0.0:8000
    I0421 21:04:17.503485 81 http_server.cc:320] Started Metrics Service at 0.0.0.0:8002
 ```


## 2. Setup directories and verify NIM endpoints

In this step, we will setup the directories and configure NIM endpoints for DiffDock inferencing.

__NOTE:__ In the following code, update the `base_url` to match with the URL of the node running DiffDock NIM.

Please make sure the required packages and dependencies are installed. You may install the required packages using the following `pip install` command.

In [1]:
# !pip install ipywidgets loguru py3dmol rdkit

In [2]:
import time
import os
import shutil
import json
import requests
from loguru import logger
import subprocess

# overwrite the output directory
def prepare_output_directory(output):
    """
    Prepare the output directory
    output: str, the output directory
    return: None
    """
    # overwrite the output directory
    # delete the output directory if it exists
    if os.path.exists(output):
        shutil.rmtree(output)
    os.makedirs(output)
    
output_dir = "./output/diffdock_result/"
prepare_output_directory(output_dir)


base_url="http://0.0.0.0:8000/"          # UPDATE the url if required
query_url = base_url + "/molecular-docking/diffdock/generate"
health_check_url = base_url + "/v1/health/ready"


### 2.1 Run Healthcheck

This is to ensure the DiffDock inference endpoint is reachable and the NIMS is up and running. It will return `true` if that is the case.

In [3]:
# check health of the DiffDock NIM endpoint
response = requests.get(health_check_url)
response.text

'true'

## 3. Perform DiffDock docking inferencing

Now that we have the DiffDock NIM up and running, and the inference endpoint is available for docking inference, we will submit a docking request.

Protein-ligand Docking request requires an input protein coordinates (in PDB format) and a ligand input file (in SDF, MOL or SMILES format).


In [4]:
# predicted target protein file path
protein_file_path = "./protein_input_file/mpro_sarscov2.pdb"

# clean SDF file directory generated from MolMIM
sdf_dir = "./ligand_input_files"

# Load and sort ligand files based on the numeric part in the filename (molecule_0, molecule_1, molecule_2 ....)
sdf_files = [f for f in os.listdir(sdf_dir) if f.endswith(".sdf")]
sdf_files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

# get name of the sdf files
ligand_names = [os.path.basename(f).split(".")[0] for f in sdf_files]

print('Input ligand files: ', sdf_files)

Input ligand files:  ['compound_1.sdf', 'compound_2.sdf', 'compound_3.sdf', 'compound_4.sdf']


In the following code block, we're defining the `run_diffdock` function that prepares the input arguments for DiffDock inference request in a JSON format, and saves the inference output received from the DiffDock NIM.

In [5]:
# preparing and formatting the input arguments for DiffDock inference request, and saving the results in output.json file
def run_diffdock(query_url, protein_file_path, ligand_file_path):
    """
    Main function to run the molecular docking
    :param query_url: str, the url to send the request to
    :param protein_file_path: str, path to the protein file
    :param ligand_file_path: str, path to the ligand file
    return JSON response
    """

    protein_bytes = file_to_json_compatible_string(protein_file_path)
    ligand_bytes = file_to_json_compatible_string(ligand_file_path)

    data = {
        "ligand": ligand_bytes,
        "ligand_file_type": "sdf",
        "protein": protein_bytes,
        "num_poses": 10,
        "time_divisions": 20,
        "steps": 18,
        "save_trajectory": False,  # diffusion trajectory
        "is_staged": False
    }

    headers = {"Content-Type": "application/json"}

    response = requests.post(query_url, headers=headers, json=data)

    if response.status_code == 200:
        print("Request successful, output saved to output.json")
    else:
        print(f"Request failed with status code {response.status_code}")
        print("Response:", response.text)

    return response.json()

# reading in the input PDB/SDF/SMILES files as a string to be used for JSON request
def file_to_json_compatible_string(file_path):
    """
    Convert PDB file and sdf file to JSON
    """
    with open(file_path, 'r') as file:
        content_str = file.read()
    return content_str

In this example, we will submit a protein PDB file (Main protease subunit of the Sars-Cov-2 virus). For input small molecules, we will use four Ensitrelvir analogs in SDF format, generated using NVIDIA MolMIM NIM.

In [6]:
# iterating over input files for DiffDock inference request submissions
for ligand_file_path, ligand_name in zip(sdf_files, ligand_names):

    start = time.time()
    
    # submitting inference request for docking pose predictions
    result = run_diffdock(
        query_url=query_url,
        protein_file_path=protein_file_path,
        ligand_file_path=os.path.join(sdf_dir, ligand_file_path),
    )
    
    end = time.time()
    logger.debug(f"{ligand_name} took {end - start:.2f} seconds")

    # save result to output.json
    ligand_output_dir = os.path.join(output_dir, ligand_name)
    prepare_output_directory(ligand_output_dir)
    with open(f"{ligand_output_dir}/output.json", "w") as f:
        json.dump(result, f)

    # save ligand positions
    for i, ligand_geometry in enumerate(result["ligand_positions"]):
        with open("{}/pose_{}.sdf".format(ligand_output_dir, i), "w") as f:
            f.write(ligand_geometry)

2024-08-02 10:24:32.235 | DEBUG    | __main__:<module>:14 - compound_1 took 2.15 seconds


Request successful, output saved to output.json


2024-08-02 10:24:34.365 | DEBUG    | __main__:<module>:14 - compound_2 took 2.13 seconds


Request successful, output saved to output.json


2024-08-02 10:24:36.689 | DEBUG    | __main__:<module>:14 - compound_3 took 2.32 seconds


Request successful, output saved to output.json


2024-08-02 10:24:38.739 | DEBUG    | __main__:<module>:14 - compound_4 took 2.05 seconds


Request successful, output saved to output.json


## 4. Visualize the docked poses

Now that we have received the DiffDock docking inference output saved as `output.json` file, we will use __`py3Dmol`__ to visualize the docking poses.

Some of the fields in the __`output.json`__ file are:
- `trajectory`: diffusion trajectory (empty unless `save_trajectory` is set to `True`)
- `ligand_positions`: a list of docking poses
- `ligand_scores`: a list of confidence scores for each docking pose
- `protein`: input protein
- `ligand`: input ligand

Confidence score the logits of the probability that the docked pose has a RMSD < 2A compared to ground truth. Interpretation of confidence score (c) is based on the guideline provided by [github authors](https://github.com/gcorso/DiffDock?tab=readme-ov-file#faq--). 
```
c > 0 : high confidence 
-1.5 < c < 0: moderate confidence 
c < -1.5: low confidence 
```


### Molecular Viewer (py3Dmol)

In the following cell, we're configuring the __py3Dmol__ for visualizing the docking poses. 

In [7]:
# Adding required libraries for an interactive protein-ligand docking visualization
# Please run the folllowing pip install command to install necessary libraries before proceeding 
# !pip install py3Dmol rdkit ipywidgets

import py3Dmol
from rdkit import Chem
import ipywidgets as widgets
from IPython.display import display
import glob
import random

# defining a function for color definitions for visualization
def ansi_color(text, color):
    """Color text for console output"""
    colors = {
        "red": "\033[31m",
        "green": "\033[32m",
        "yellow": "\033[33m",
        "blue": "\033[34m",
        "magenta": "\033[35m",
        "cyan": "\033[36m",
        "white": "\033[37m",
        "reset": "\033[0m"
    }
    return f"{colors[color]}{text}{colors['reset']}"

# loading dock poses from the output SDF files extracted from the output.json 'positions' field
def load_poses_from_sdf(directory):
    sdf_files = glob.glob(f"{directory}/*.sdf")
    poses = []
    
    for sdf_file in sdf_files:
        supplier = Chem.SDMolSupplier(sdf_file)
        for mol in supplier:
            if mol is not None:
                poses.append(mol)  
    return poses

# visualising the docking poses in an interactive manner, browsing docked poses using an embedded slider
def update_viewer(pose_index):
    
    view = py3Dmol.view(width=1200, height=900)
    
    # Add the protein model
    view.addModel(protein_pdb, 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'white', 'opacity': 0.7}})
    view.setViewStyle({'style':'outline','color':'black','width':0.03})
    Prot=view.getModel()
    Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.4,'color':'white'})
    
    # Add the selected docking pose
    pose = poses[pose_index]
    pose_block = Chem.MolToMolBlock(pose)
    # color = "#"+''.join([random.choice('0123456789ABCDEF') for _ in range(6)])
    view.addModel(pose_block, 'mol')
    view.setStyle({'model': 1}, {'stick': {'radius': 0.3, 'colorscheme': 'magentaCarbon'}})
    view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': 'magentaCarbon'}, {'model': 1})
    score = round(confidence_scores[pose_index], 3)
    score_color = "green" if score > -0.5 else "blue" if score >= -1.5 else "red"
    print(f"Loaded {ansi_color(ligand_name, 'magenta')} with confidence score: {ansi_color(confidence_scores[pose_index], score_color)}")
    view.zoomTo()
    return view.update()

# Load the protein model
with open(protein_file_path, 'r') as f:
    protein_pdb = f.read()

# Specify the directory containing the dock poses in SDF format for a specific ligand
ligand_name = "compound_1"
directory = output_dir + ligand_name
poses = load_poses_from_sdf(directory)

# Verify the number of poses loaded
print(f"Number of poses loaded: {len(poses)}")

# Load confidence scores from output.json
output_json_path = os.path.join(directory, 'output.json')
with open(output_json_path, 'r') as file:
    data = json.load(file)
    confidence_scores = data['position_confidence']  # list of floats

# Create a slider widget
pose_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(poses) - 1,
    step=1,
    description='Pose:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Link the slider to the viewer update function
widgets.interact(update_viewer, pose_index=pose_slider)

Number of poses loaded: 10


interactive(children=(IntSlider(value=0, continuous_update=False, description='Pose:', max=9), Output()), _dom…

<function __main__.update_viewer(pose_index)>